In [1]:
from linares_plot import *
import sys, os
path_tools = 'C:\\Users\\David\\Documents\\GitHub\\encoding\\scripts\\wm_representation\\functions\\IEM\\tools'
sys.path.insert(1, path_tools)
from tiemcourse_interference import *
from scipy import stats
from scipy.stats import norm

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
df = pd.read_excel('C:\\Users\\David\\Desktop\\KI_Desktop\\data_reconstructions\\IEM\\IEM_trainT_decoded_angle.xlsx')
len(df)

89856

In [4]:
def err_deg(a1,ref):
    ### Calculate the error ref-a1 in an efficient way in the circular space
    ### it uses complex numbers!
    ### Input in degrees (0-360)
    a1=np.radians(a1)
    ref=np.radians(ref)
    err = np.angle(np.exp(1j*ref)/np.exp(1j*(a1) ), deg=True) 
    err=round(err, 2)
    return err

In [ ]:
error_ = [err_deg(df.iloc[i].decoded_angle, df.iloc[i].target_centered) for i in range(len(df)) ]
distr_distance = [err_deg(df.iloc[i].distractor_centered, df.iloc[i].target_centered) for i in range(len(df)) ]

df['error'] = error_
df['error_abs'] = abs(np.array(error_))
df['distractor_distance'] = distr_distance
df['distractor_distance_abs'] = abs(np.array(distr_distance))

df['sign_dist'] = df.distractor_distance<0
df['sign_dist'] = df['sign_dist'].replace([True, False], ['ccw', 'cw']) #position of the distractor relative to the target

C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in cdouble_scalars
  import sys


In [ ]:
df_beh = pd.read_excel('C:\\Users\\David\\Desktop\\KI_Desktop\\data_reconstructions\\IEM\\IEM_trainT_decoded_angle_related_behaviour.xlsx')

In [ ]:
df_beh.head()

In [ ]:
len(df_beh)

In [ ]:
df.head(5)